<a href="https://colab.research.google.com/github/SEEsuite/colab_scripts/blob/main/regenerate_instagram_image_urls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get temp instagram urls.

Most instagram posts have more than one image associated with it. THis script will duplicate the original post row for each image in the post. (You will leave with a longer excel sheet than you started with)


This should take about 30 minutes per 1000 instance. You are free to decrease the sleep range in random.choice - I really have no clue if instagram would boot you for accessing posts faster than the speed of light.

If you don't want to have to keep regenerating the link, use `wget` to download images for longterm storage

In [ ]:
!pip install wget

In [4]:
import re
import json

import pandas as pd
import time
from json import JSONDecodeError
import random
import requests

import wget

In [5]:
import urllib.request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def import_data_from_drive(share_link, your_name_for_file="my_data"):
  """Brings data file from a google drive sharepoint to your colab workspace.
     It does not require you to host the dataset on your own account.

     Parameters:
     share_link: the link to view a file in google drive
     our_name_for_file: a string describing the file, preferable endling in a file type, ex. 'data.csv'
     """
  id = share_link.split("/")[5] # separate the id from the link
  print("Using id", id, "to find file on drive")

  # use pydrive and colab modules to authenticate you
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Authenticated colab user")

  # This step will move the file from Drive to the workspace
  download_object = drive.CreateFile({'id':id}) 
  download_object.GetContentFile(your_name_for_file)
  print("Added file to workspace with name", your_name_for_file)

  return

In [6]:
# YOU CHANGE HERE

link = "https://docs.google.com/spreadsheets/d/12piHij4GRIi8L14IFF57p7RWtOkthC39/edit?usp=sharing&ouid=101042095541764641159&rtpof=true&sd=true"

import_data_from_drive(link, your_name_for_file="influencers.xlsx")

df = pd.read_excel("influencers.xlsx")

Using id 12piHij4GRIi8L14IFF57p7RWtOkthC39 to find file on drive
Authenticated colab user
Added file to workspace with name influencers.xlsx


In [7]:
df.columns

Index(['Unnamed: 0', 'Query Id', 'Query Name', 'Date', 'Title', 'Snippet',
       'Full Text', 'Url', 'Domain', 'Sentiment', 'Emotion', 'Page Type',
       'Language', 'Country Code', 'Continent Code', 'Continent', 'Country',
       'Region Code', 'Region', 'City Code', 'Account Type', 'Assignment',
       'Author', 'Avatar', 'Category Details', 'Checked', 'City',
       'Display URLs', 'Expanded URLs', 'Facebook Author ID',
       'Facebook Comments', 'Facebook Likes', 'Facebook Role',
       'Facebook Shares', 'Facebook Subtype', 'Full Name', 'Gender',
       'Hashtags', 'Impact', 'Impressions', 'Instagram Comments',
       'Instagram Followers', 'Instagram Following', 'Instagram Likes',
       'Instagram Posts', 'Interest', 'Last Assignment Date', 'Latitude',
       'Location Name', 'Longitude', 'Media Filter', 'Media URLs',
       'Mentioned Authors', 'Priority', 'Professions', 'Resource Id',
       'Short URLs', 'Starred', 'Status', 'Subtype', 'Tags', 'Thread Author',
       'Thre

In [8]:
def get_instance(ob, x):
    url = re.sub(r"\\/", "/", ob['url'])
    instance = x.copy().to_frame().T
    instance['temporary_urls'] = url
    # print(x)
    # print(instance.head())
    return instance
    # urls_to_post_images.append(url)


def handle_unknown_error(url, e):
  print("something went wrong on url", url ) 
  print("error:", e)
  error_urls.append((url, e))
  return


In [9]:

error_urls = []
subset = df.loc[df['Subtype'] == 'photo']## excluding non-image posts
rows = pd.DataFrame(columns=subset.columns)

# note - its not a good habit to iterate over a dataframe like this. its slow. But in this case we don't mind slow,
# because there is a chance instagram will rate-limit us.
for i, x in subset.iterrows(): 
  print()
  post_url = x['Url']
  print(post_url)
  urls_to_post_images = []
  # url = "https://www.instagram.com/p/Cm1fufQL0zR/"

  try:

    payload = requests.get(post_url)

    #some json nonsense. pls ignore. If this is breaking its likely that the payload has changed.
    l = re.findall('"image":(\[\{[^\]]*)', str(payload.content))[0]
    l += "]"
    links = json.loads(l)


    print(f"post returned {len(links)} images")
    for ob in links:
      row = get_instance(ob, x)
      rows = pd.concat([rows, row], axis=0)

  except JSONDecodeError:
    try: 
      print("JSON DECODE ERROR, removing caption to hopefully get rid of any wierd symbols")
      l = re.sub('caption":.*","representativeOfPage' , 'representativeOfPage', l)
      print(l)
      links = json.loads(l)
      print(f"post returned {len(links)} images")

      for ob in links:
        row = get_instance(ob, x)
        rows = pd.concat([rows, row], axis=0)
    except Exception as e:
      handle_unknown_error(post_url, e)
  except Exception as e:
    handle_unknown_error(post_url, e)
  # break
  time.sleep(random.choice([1,1.5,2])) # I'm such a sneaky hacker


https://www.instagram.com/p/Cm2mIrYrUJb/
post returned 1 images

https://www.instagram.com/p/Cm2JHTGPSKT/
post returned 9 images

https://www.instagram.com/p/Cm1fufQL0zR/
JSON DECODE ERROR, removing caption to hopefully get rid of any wierd symbols
[{"\\u0040type":"https:\\/\\/schema.org\\/ImageObject","representativeOfPage":"True","height":"1440","width":"1440","url":"https:\\/\\/scontent-sea1-1.cdninstagram.com\\/v\\/t51.29350-15\\/322674492_151097997688850_7757197253696482083_n.jpg?stp=dst-jpg_s640x640&_nc_cat=105&ccb=1-7&_nc_sid=8ae9d6&_nc_ohc=7-TuTTauLagAX8yZ9I5&_nc_ht=scontent-sea1-1.cdninstagram.com&oh=00_AfDbA2nWeBKvxQ1XJ-iLMSgbGnCTaQpW9WfRPHmGkSvX2w&oe=64604B25"}]
post returned 1 images

https://www.instagram.com/p/CmyyI8crypt/
post returned 1 images

https://www.instagram.com/p/Cmx0u10u_n1/
something went wrong on url https://www.instagram.com/p/Cmx0u10u_n1/
error: list index out of range

https://www.instagram.com/p/CmwY-M1rnfX/
post returned 1 images

https://www.instagram

In [11]:
rows.drop(columns=['Unnamed: 0']).reset_index(inplace=True)
rows

,Unnamed: 0,Query Id,Query Name,Date,Title,Snippet,Full Text,Url,Domain,Sentiment,...,Reach (new),Reddit Score,Reddit Score Upvote Ratio,Reddit Author Karma,Reddit Author Awardee Karma,Reddit Author Awarder Karma,Reddit Comments,Subreddit,Subreddit Subscribers,temporary_urls
0,1,2001569258,Sponsored IG test,2023-01-01 00:50:40.000,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,https://www.instagram.com/p/Cm2mIrYrUJb/,instagram.com,neutral,...,380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...
2,3,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...
2,3,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...
2,3,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...
2,3,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342,1343,2001569258,Sponsored IG test,2022-01-02 19:47:14.000,have the holidays stressed you out? are you si...,...#sundayscaries #goth #gothgoth #sponsored #...,have the holidays stressed you out? are you si...,https://www.instagram.com/p/CYPW2pDvrOe/,instagram.com,negative,...,4081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...
1343,1344,2001569258,Sponsored IG test,2022-01-02 11:43:01.000,A volte sono anche vestito 😌🤜🏻 - - - - - - - -...,A volte sono anche vestito 😌🤜🏻 - - - - - - - -...,A volte sono anche vestito 😌🤜🏻 - - - - - - - -...,https://www.instagram.com/p/CYOfcGLs8__/,instagram.com,neutral,...,4304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...
1344,1345,2001569258,Sponsored IG test,2022-01-02 04:57:29.000,Come to my aid babies! 😢 I'm waiting for you i...,...#lgbt #following #toesucking #americanguys ...,Come to my aid babies! 😢 I'm waiting for you i...,https://www.instagram.com/p/CYNxB2XKj7-/,instagram.com,positive,...,574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...
1345,1346,2001569258,Sponsored IG test,2022-01-02 03:51:47.000,This New Years hangover is #sponsored by heyya...,This New Years hangover is #sponsored by heyya...,This New Years hangover is #sponsored by heyya...,https://www.instagram.com/p/CYNpgrzLiSN/,instagram.com,neutral,...,2399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...


In [12]:
# most of the error list is dead posts but saving them just in case you want to investigate
import pickle

with open("error_list", 'wb') as f:
    pickle.dump(error_urls, f)

# probably don't investigate if you are not comfortable working with pickle, list of tuples, etc.

In [13]:
def app(url):
  return url.strip(']\'[').split(', ')

rows["temporary_urls"] = rows["temporary_urls"].apply(app)

# each row should only have one now... 

In [14]:
rows['temporary_urls']

0       [https://scontent-sea1-1.cdninstagram.com/v/t5...
2       [https://scontent-sea1-1.cdninstagram.com/v/t5...
2       [https://scontent-sea1-1.cdninstagram.com/v/t5...
2       [https://scontent-sea1-1.cdninstagram.com/v/t5...
2       [https://scontent-sea1-1.cdninstagram.com/v/t5...
                              ...                        
1342    [https://scontent-sea1-1.cdninstagram.com/v/t5...
1343    [https://scontent-sea1-1.cdninstagram.com/v/t5...
1344    [https://scontent-sea1-1.cdninstagram.com/v/t5...
1345    [https://scontent-sea1-1.cdninstagram.com/v/t5...
1346    [https://scontent-sea1-1.cdninstagram.com/v/t5...
Name: temporary_urls, Length: 1505, dtype: object

In [16]:
rows.to_excel("instagram_images_with_live_links_MAY10.xlsx")

In [31]:
!mkdir /content/image_folder
# !rm  /content/image_folder/*

In [ ]:

# Option to store long-term (still need to download from colab)

#acould lso just run this in a local pyton environment.


for i, media_file in enumerate(rows['temporary_urls']):
  wget.download(media_file[0], out =f"/content/image_folder/image{i}.png")




In [ ]:
!zip -r /content/image_folder.zip /content/image_folder/

#NOW GO DOWNLOAD THE IMAGE FOLDER.zip